In [ ]:
%%writefile final.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import io
# :estrella2: Configuración de la aplicación
st.set_page_config(page_title=":gráfico_de_barras: Análisis de Ventas", layout="wide")
st.title(":gráfico_de_barras: Análisis de Ventas")
# :carpeta_abierta: Subida de archivo (CSV o Excel)
archivo = st.file_uploader(":carpeta_abierta: Sube un archivo CSV o Excel", type=["csv", "xlsx"])
if archivo:
    # :chincheta: Detectar formato y leer archivo
    if archivo.name.endswith(".csv"):
        df = pd.read_csv(archivo)
    elif archivo.name.endswith(".xlsx"):
        df = pd.read_excel(archivo, engine="openpyxl")  # Para compatibilidad con Excel moderno
    # :chincheta: Convertir Fecha a formato datetime
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    
#EJERCICIO 2
#filtro por producto
producto_seleccionado = st.selectbox("Selecciona un producto:", df["Producto"].unique())
df_filtrado = df[df["Producto"] == producto_seleccionado]
st.write(producto_seleccionado)
st.dataframe(df_filtrado)

#Gráfico de ventas por fecha
st.write(":calendar: Ventas a lo largo del tiempo:")
# Extraer el año y mes de la columna 'Fecha' para crear una nueva columna
df["Mes-Año"] = df["Fecha"].dt.to_period("M").dt.strftime('%Y-%m')  # Convertimos a string "YYYY-MM"
ventas_por_mes = df.groupby("Mes-Año")["Total"].sum()  # Agrupar por Mes-Año
# Mostrar gráfico de líneas con la evolución de las ventas por mes
st.line_chart(ventas_por_mes)

# :gráfico_con_tendencia_ascendente: Histograma de Precios Unitarios
st.subheader(":bolsa_de_dinero: Distribución de Precios Unitarios")
fig, ax = plt.subplots()
ax.hist(df["Precio Unitario"], bins=10, color="skyblue", edgecolor="black")
st.pyplot(fig)

# :calendario: Resumen mensual de ventas
st.subheader(":calendario: Resumen de Ventas por Mes")
df["Mes-Año"] = df["Fecha"].dt.to_period("M").dt.strftime('%Y-%m')  # Crear una nueva columna con el mes y el año como string
ventas_por_mes = df.groupby("Mes-Año")["Total"].sum()
st.bar_chart(ventas_por_mes)

# :trofeo: Top 5 de productos más vendidos
st.subheader(":trofeo: Top 5 de Productos Más Vendidos")
top_productos = df.groupby("Producto")["Total"].sum().sort_values(ascending=False).head(5)
st.table(top_productos)

# :fecha: Filtro de fechas
st.subheader(":fecha: Filtrar Ventas por Rango de Fechas")

# Selección de fechas con corrección
min_fecha = df["Fecha"].min().date()
max_fecha = df["Fecha"].max().date()
fecha_inicio, fecha_fin = st.date_input(
    ":calendario: Selecciona un rango de fechas",
    (min_fecha, max_fecha)
)

# Convertir a datetime64[ns] antes de filtrar
fecha_inicio = pd.to_datetime(fecha_inicio)
fecha_fin = pd.to_datetime(fecha_fin)
df_filtrado_fechas = df[(df["Fecha"] >= fecha_inicio) & (df["Fecha"] <= fecha_fin)]
st.write(f":calendario: Datos entre {fecha_inicio.date()} y {fecha_fin.date()}")
st.dataframe(df_filtrado_fechas)

# :bandeja_de_entrada: Botón para descargar datos filtrados en CSV
st.subheader(":bandeja_de_entrada: Descargar Datos Filtrados")
csv = df_filtrado_fechas.to_csv(index=False).encode("utf-8")
st.download_button(label=":bandeja_de_entrada: Descargar CSV", data=csv, file_name="datos_filtrados.csv", mime="text/csv")

# :bandeja_de_entrada: Botón para descargar datos filtrados en Excel
output = io.BytesIO()
with pd.ExcelWriter(output, engine="xlsxwriter") as writer:
    df_filtrado_fechas.to_excel(writer, index=False, sheet_name="Datos Filtrados")
    writer.close()
st.download_button(
    label=":bandeja_de_entrada: Descargar Excel",
    data=output.getvalue(),
    file_name="datos_filtrados.xlsx",
    mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
)